In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from delta.tables import *
from pyspark.sql.functions import *


StatementMeta(, 163b879d-ad53-4620-a4d6-068e682532b0, 3, Finished, Available)

In [2]:
lakehousePath = "abfss://85bfc254-9abf-46cc-b1fe-943ec35b3460@msit-onelake.dfs.fabric.microsoft.com/c02dea28-20ca-432b-b6e8-39d0be76f540"
tableName = "Invoices"
tableKey = "InvoiceID"
dateColumn = "LastEditedWhen"

StatementMeta(, 163b879d-ad53-4620-a4d6-068e682532b0, 4, Finished, Available)

In [3]:
deltaTablePath = f"{lakehousePath}/Tables/{tableName}" 
# print(deltaTablePath)

StatementMeta(, 163b879d-ad53-4620-a4d6-068e682532b0, 5, Finished, Available)

In [9]:
parquetFilePath = f"{lakehousePath}/Files/incremental/{tableName}/{tableName}.parquet"
# print(parquetFilePath)

StatementMeta(, e3bfaad7-54ed-4f11-a2bb-307e2a9d49f3, 11, Finished, Available)

In [10]:
df2 = spark.read.parquet(parquetFilePath)
# display(df2)

StatementMeta(, e3bfaad7-54ed-4f11-a2bb-307e2a9d49f3, 12, Finished, Available)

Check if table already exists; if it does, do an upsert and return how many rows were inserted and update; if it does not exist, return how many rows were inserted

In [11]:
if DeltaTable.isDeltaTable(spark,deltaTablePath):
    deltaTable = DeltaTable.forPath(spark,deltaTablePath)
    deltaTable.alias("t").merge(
        df2.alias("s"),
        f"t.{tableKey} = s.{tableKey}"
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    history = deltaTable.history(1).select("operationMetrics")
    operationMetrics = history.collect()[0]["operationMetrics"]
    numInserted = operationMetrics["numTargetRowsInserted"]
    numUpdated = operationMetrics["numTargetRowsUpdated"]
else:
    df2.write.format("delta").save(deltaTablePath)  
    deltaTable = DeltaTable.forPath(spark,deltaTablePath)
    operationMetrics = history.collect()[0]["operationMetrics"]
    numInserted = operationMetrics["numTargetRowsInserted"]
    numUpdated = 0

StatementMeta(, e3bfaad7-54ed-4f11-a2bb-307e2a9d49f3, 13, Finished, Available)

Get the latest date loaded into the table - this will be used for watermarking; return the max date, the number of rows inserted and number updated

In [6]:
deltaTablePath = f"{lakehousePath}/Tables/{tableName}"
df3 = spark.read.format("delta").load(deltaTablePath)
maxdate = df3.agg(max(dateColumn)).collect()[0][0]
# print(maxdate)
maxdate_str = maxdate.strftime("%Y-%m-%d %H:%M:%S")

StatementMeta(, 163b879d-ad53-4620-a4d6-068e682532b0, 8, Finished, Available)

In [ ]:
result = "maxdate="+maxdate_str +  "|numInserted="+str(numInserted)+  "|numUpdated="+str(numUpdated)
# result = {"maxdate": maxdate_str, "numInserted": numInserted, "numUpdated": numUpdated}
mssparkutils.notebook.exit(str(result))